In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark import SparkContext

In [4]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [5]:
sc = SparkContext()

In [6]:
ssc = StreamingContext(sc,10)
SQLContext = SQLContext(sc)

C:\spark-3.3.0-bin-hadoop2\python\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [16]:
socket_stream = ssc.socketTextStream('127.0.0.1', 9999)

Py4JJavaError: An error occurred while calling o23.socketTextStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after stopping a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:230)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:67)
	at org.apache.spark.streaming.dstream.InputDStream.<init>(InputDStream.scala:45)
	at org.apache.spark.streaming.dstream.ReceiverInputDStream.<init>(ReceiverInputDStream.scala:42)
	at org.apache.spark.streaming.dstream.SocketInputDStream.<init>(SocketInputDStream.scala:40)
	at org.apache.spark.streaming.StreamingContext.socketStream(StreamingContext.scala:321)
	at org.apache.spark.streaming.StreamingContext.$anonfun$socketTextStream$1(StreamingContext.scala:302)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.streaming.StreamingContext.withNamedScope(StreamingContext.scal

In [8]:
lines = socket_stream.window(20)

In [9]:
from collections import namedtuple
fields = ('tag', 'count')
Tweet = namedtuple( 'Tweet', fields)

In [10]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) ) # Registers to a table.

In [11]:
ssc.start()    

In [12]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [14]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = SQLContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    sns.plt.show()
    count = count + 1

AnalysisException: Table or view not found: tweets; line 1 pos 23;
'Project ['tag, 'count]
+- 'UnresolvedRelation [tweets], [], false


In [15]:
ssc.stop()